# Imports

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,\
                                    cross_validate,\
                                    GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import LabelEncoder,\
                                  StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

# Load data

In [23]:
dataset = pd.read_csv("Churn Modeling.csv")
dataset.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [24]:
y = dataset["Exited"]
X = dataset.loc[:, :"EstimatedSalary"]

In [25]:
col_to_encode = ["Geography", "Gender"]
encoder = LabelEncoder()
for col in col_to_encode:
    X[col] = encoder.fit_transform(X[col])

In [26]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Build pipeline

In [28]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ("ada_clf", AdaBoostClassifier())])

pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.86

In [29]:
prediction = pipe.predict(X_test)
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      2379
           1       0.73      0.52      0.60       621

    accuracy                           0.86      3000
   macro avg       0.81      0.73      0.76      3000
weighted avg       0.85      0.86      0.85      3000



# Hyperparameter tuning

In [37]:
parameters = {
    "base_estimator": [LogisticRegression(), DecisionTreeClassifier(), SVC()],
    "n_estimators": [10, 30, 50, 70, 100],
    "learning_rate": [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5],
    "algorithm": ["SAMME", "SAMME.R"]
}

clf_optim = GridSearchCV(estimator=AdaBoostClassifier(random_state=0),
            param_grid=parameters)

In [ ]:
clf_optim.fit(X_train, y_train)